Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to boolean data type |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [2]:
import pandas as pd
import numpy as np

/var/folders/_w/7dw428vj2p901s_9nz9060_80000gn/T/ipykernel_35986/2578680215.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [84]:
df = pd.read_csv('bank_marketing.csv')

In [77]:
client_data = df[["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]]

client_data.loc[:, "job"] = client_data["job"].str.replace(".", "_")
client_data.loc[:, "education"] = client_data["education"].str.replace(".", "_")
client_data.loc[:, "education"] = client_data["education"].replace('unknown', np.nan)
client_data.loc[:,'credit_default'] = client_data['credit_default'].replace({'yes': True, 'no': False, 'unknown': np.nan})
client_data.loc[:,'mortgage'] = client_data['mortgage'].replace({'yes': True, 'no': False, 'unknown': np.nan})

client_data['credit_default'] = client_data['credit_default'].astype(bool)
client_data['mortgage'] = client_data['mortgage'].astype(bool)

/var/folders/_w/7dw428vj2p901s_9nz9060_80000gn/T/ipykernel_35986/3960226473.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_data['credit_default'] = client_data['credit_default'].astype(bool)
/var/folders/_w/7dw428vj2p901s_9nz9060_80000gn/T/ipykernel_35986/3960226473.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client_data['mortgage'] = client_data['mortgage'].astype(bool)


In [78]:
campaign = df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]

campaign.loc[:,'previous_outcome'] = campaign['previous_outcome'].replace({'success': True, 'failure': False, 'nonexistent': np.nan})
campaign.loc[:, 'campaign_outcome'] = campaign['campaign_outcome'].replace({"yes": True, "no": False})

campaign['previous_outcome'] = campaign['previous_outcome'].astype(bool)
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype(bool)

campaign['last_contact_date'] = pd.to_datetime('2022' + df['month'].astype(str) + df['day'].astype(str), format='%Y%b%d')

/var/folders/_w/7dw428vj2p901s_9nz9060_80000gn/T/ipykernel_35986/35875309.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  campaign.loc[:, 'campaign_outcome'] = campaign['campaign_outcome'].replace({"yes": True, "no": False})
/var/folders/_w/7dw428vj2p901s_9nz9060_80000gn/T/ipykernel_35986/35875309.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign['previous_outcome'] = campaign['previous_outcome'].astype(bool)
/var/folders/_w/7dw428vj2p901s_9nz9060_80000gn/T/ipykernel_35986/35875309.py:7: SettingWithCopyW

In [81]:
economics = pd.DataFrame()
economics["client_id"] = df["client_id"]

empty_list = [None] * len(df)
economics["cons_price_idx"] = empty_list
economics["euribor_three_months"] = empty_list

economics['cons_price_idx'] = economics['cons_price_idx'].astype(float)
economics['euribor_three_months'] = economics['euribor_three_months'].astype(float)

print(economics)

       client_id  cons_price_idx  euribor_three_months
0              0             NaN                   NaN
1              1             NaN                   NaN
2              2             NaN                   NaN
3              3             NaN                   NaN
4              4             NaN                   NaN
...          ...             ...                   ...
41183      41183             NaN                   NaN
41184      41184             NaN                   NaN
41185      41185             NaN                   NaN
41186      41186             NaN                   NaN
41187      41187             NaN                   NaN

[41188 rows x 3 columns]


In [83]:
client_data.to_csv('client_data.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)